In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("LANGCHAIN_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Self Improvement"

# Tools

In [3]:
from utils.tools import construct_tools, get_tools_descriptions
from langgraph.prebuilt import ToolNode

tools = construct_tools()
tools_descriptions = get_tools_descriptions(tools)
tool_node = ToolNode(tools)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", base_url="https://api.chsdw.top/v1")

# Define State

In [31]:
from typing import Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

MAX_ITERATIONS = 4

class State(TypedDict):
    input: str
    critique: Annotated[list[str], add]
    react_messages: Annotated[list, add]
    final_answer: Annotated[list[str], add]
    iteration: int

# Define Reacter

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage,HumanMessage

react_prompt = ChatPromptTemplate.from_messages([
    ("system", "Identify all the candidate tools you may need to use and the corresponding tool input text based on the user's current step. "
     "Always remmeber that there may be multiple tools that can be used to complete a step! So if there are more than one tool that can be used, "
     "your response should contain multiple tools and their inputs. Make sure that the current step is completed before moving to the next step. ")
    #  "You have access to the following tools:\n\n"
    #  "{tools_descriptions}\n\n")
     ])

react_prompt = react_prompt.format(tools_descriptions=tools_descriptions)

reacter = create_react_agent(model=llm, tools=tools, state_modifier=SystemMessage(content=react_prompt))

def react(state):
    if state["critique"]:
        critique = state["critique"][-1]
        inputs = {"messages": [("user", f"{critique}\n Based on the previous critique, answer the question. "),("user", state["input"])]}
        result = reacter.invoke(input=inputs)
    else:
        inputs = {"messages": [("user", state["input"])]}
        result = reacter.invoke(input=inputs)
    return {
        "react_messages": [result["messages"]]
    }

# Define Critic

In [80]:
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

critic_prompt = ("Inspect the previous messages and identify any potential issues or errors. "
                 "Is the selection of tools calling reasnonable? "
                 "Check the input to the tools step by step. "
                 "Is the final answer truthful?"
                 "And give a better solution or input to the tools. \n"
                 "Your response should be no more than 250 words. ")

final_answer_prompt = ("Based on the conversation, provide the final answer to the user's question. "
                       "The final answer should be a single number. Follow the format: \n"
                       "FINAL ANSWER: <answer> \n")

def criticize(state):
    if state["react_messages"]:
        react_messages = state["react_messages"][-1]
        critique = llm.invoke(react_messages + [HumanMessage(content=critic_prompt)])
        final_answer = llm.invoke(react_messages + [HumanMessage(content=final_answer_prompt)])
        return {
            "critique": [critique.content],
            "final_answer": [final_answer.content.split("FINAL ANSWER:")[-1].strip()],
            "iteration": 1 if not state["iteration"] else state["iteration"] + 1 
        }
    else:
        return {
            "critique": ["None"],
            "final_answer": ["None"],
            "iteration": 1 if not state["iteration"] else state["iteration"] + 1 
        }
    
# Either agent can decide to end
from typing import Literal

def should_end(state) -> Literal["react", "__end__"]:
    if state["iteration"] >= MAX_ITERATIONS or len(state["final_answer"]) > 1 and state["final_answer"][-1] == state["final_answer"][-2]:
        return "__end__"
    else:
        return "react"


# Construct Graph

In [81]:
builder = StateGraph(State)

builder.add_node("react", react)
builder.add_node("critic", criticize)

builder.add_edge(START, "react")
builder.add_edge("react", "critic")

builder.add_conditional_edges("critic", should_end)

graph = builder.compile()